<a href="https://colab.research.google.com/github/dhvillegas/GeminiAI_AvaliacaoPerfilLinkedIn/blob/main/Projeto_GeminiAI_AvaliacaoPerfilLinkedIn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Projeto**: Avaliação do Perfil do LinkedIn com Google Gemini  
**Problema**: Atualizar o perfil do LinkedIn pode ser um processo frustrante e demorado, especialmente para usuários que não estão familiarizados com a plataforma e não disponibilizam de tempo consequentemente um perfil desatualizado pode prejudicar a imagem profissional do usuário e dificultar a busca por novas oportunidades de trabalho.  
**Outcome**: Reduzir o tempo para atualizar o perfil, melhorar a visualização e network no LinkedIn, aumentar as oportunidades de oferta de emprego  
**Solução**: Com o arquivo gerado de Resumo (CV) no formato de PDF é póssivel fazer uma analise e receber feedbacks de melhorias nos pontos especificos de acordo com a sua vaga, o arquivo é estruturado e analisado pelo Gemini e calibrando o resultado esperado usando a engenharia de prompt **Few-shot Chain-of-Thought prompting** onde é capturado insights importantes para atualizar o perfil.  


# Configura o ambiente para rodar a solução

In [11]:
!pip install -q -U google-generativeai
!pip install -q -U PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.7 MB/s eta 0:00:00


In [98]:
#Importações e configurações
import numpy as np
import pandas as pd
import PyPDF2 as pdf
import json
import google.generativeai as genai
from google.colab import userdata
from google.colab import files

api_key = userdata.get("SECRET_KEY")
genai.configure(api_key=api_key)

In [13]:
generation_config = {
   "candidate_count": 1,
   "temperature": 0.5,
}

In [14]:
safety_settings = {
    "HARASSMENT": "BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE"
}

# Instanciando o projeto

In [15]:
model = genai.GenerativeModel(model_name="gemini-1.0-pro",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

**Atribuindo o arquivo de Resumo do LinkedIn em PDF**  
Gerar o arquivo:  
1. Acessar sua conta do LinkedIn no seu perfil ex: https://www.linkedin.com/in/meunome/  
2. Abaixo do nome e profissão, clique no botão "More"  
3. Selecione a opção Save to PDF  

Depois que o arquivo for gerado, fazer upload no Colab e na pasta Files do menu direito, apos upload clique no icone dos 3 pontos para selecionar a opção "Copy path" referente ao arquivo desejado.  
Ex: /content/Profile.pdf   
Informe o caminho do path no Prompt abaixo

In [121]:

profile_path = input("Informe o caminho da URL com o arquivo de Resumo do LinkedIn em PDF")


Informe o caminho da URL com o arquivo de Resumo do LinkedIn em PDF/content/Themis.pdf


In [122]:
# Abrir o arquivo PDF
with open(profile_path, 'rb') as arquivo_pdf:
    # Criar um objeto PdfFileReader
    leitor_pdf = pdf.PdfReader(arquivo_pdf)

    # Obter o número de páginas
    numero_paginas = len(leitor_pdf.pages)

# Extrair texto de cada página
    input_avaliar = ""
    for pagina in range(numero_paginas):
        # Obter o objeto Page
        pagina_obj = leitor_pdf.pages[pagina]

        # Extrair o texto da página
        input_avaliar = input_avaliar + pagina_obj.extract_text()

        # Processar o texto extraído (por exemplo, imprimir ou salvar em um arquivo)
    #print(input_avaliar)



# Treinando Gemini utilizando o modelo Few-shot Chain-of-Thought prompting


In [123]:
#Exemplo Modelador 1
input_modelador_1 = """Contact
email@gmail.com
www.linkedin.com/in/1234556666
(LinkedIn)
Top Skills
Dashboards
Solved Problems
Comunications
Time workFulano de Souza
Backend Developer
Rio de Janeiro
Summary
I am a passionate developer with 10 years of experience in developing robust and scalable applications, using technologies such as NodeJS, Python, MySQL, PostgreSQL, MongoDB, among others.
Currently, I work on projects where I am responsible for creating and maintaining dynamic, scalable and secure web applications.
My plurality of technologies is a strength that I have developed over the years, allowing me to contribute to the success of my clients and partners. I am always looking for new challenges and learning opportunities.
Experience
Webcore Games
Software Engineer
Mar 2022 - Present · 2 yrs 3 mos
São Paulo, São Paulo, Brasil · Remote
Desenvolvimento e manutenção de aplicações B2B e B2C, sendo a maioria projetadas para campanhas publicitárias com alto volume de usuários.
Gorila
Engenheiro de software
Jan 2021 - Feb 2022 · 1 yr 2 mos
São Paulo, São Paulo, Brasil · Remote
Trabalhei desenvolvendo as funcionalidades direcionadas aos clientes B2B, integrando com os outros serviços Gorila e externos.
Sinergia Studios
Engenheiro de software líder
Feb 2019 - Jan 2021 · 2 yrs
São Paulo e Região, Brasil
Desenvolvimento da aplicação em servidor, gerenciamento dos servidores, banco de dados e painel interno feito em angular.
Education
Impacta Tecnologia
Bacharelado, Gestão de Sistemas de Informação
ETEC Professor Camargo Aranha
Ensino Técnico, Informática"""

#Exemplo Modelador 2
input_modelador_2 = """Contato
novoemail@gmail.com
www.linkedin.com/in/33333-797bb8143  (LinkedIn)
Principais competências
Comunicação
Liderança
Data-drivenCiclana Moreira Silva
Analista de marketing sênior
Experiência
TOTVS
Analista de marketing
Jun 2021 - Present · 3 yrs
São Paulo, Brasil
Sebrae-SP
Analista comunicação e marketing
Sep 2012 - Jun 2021 · 8 yrs 10 mos
Formação acadêmica
FIAP
Digital Analytics, Marketing Digital
FIAP
UX"""


#Exemplo Modelador 3
input_modelador_3 = """Contato
novoemail@gmail.com
www.linkedin.com/in/45675reeee  (LinkedIn)
Principais competências
organização
Focado
AutodidataMarkato do Brasil
Front-end Developer | Angular
Resumo
Olá, sou Desenvolvedor Front-end graduado em Sistemas de Informação, com 4 anos de experiência em criação de sites.
Acredito na importância da usabilidade e acessibilidade, por isso, me dedico a construir interfaces responsivas e intuitivas, sempre buscando a melhor experiência para o usuário.
Experiência
Lejour
Desenvolvedor web
Mar 2020 - Feb 2024 · 4 yrs
ão Paulo, Brasil · Remote
- Desenvolvimento de dashboards internos e externos utilizando Angular (sempre na última versão);
- Participação na criação do design system com lib angular que contém e exporta estilos, componentes e serviços que serão reutilizados através dos diferentes dashboards;
- Desenvolvimento orientado a testes (TDD) em todos os projetos.
Insize Brasil
Auxiliar administrativo
Jan 2015 - Oct 2015 · 10 mos
São Paulo, Brasil · On-site
Atendimento ao cliente, participação em licitações eletrônicas, elaboração e envio de propostas.
Formação acadêmica
Centro Universitário Senac
Analise e Desenvolvimento de Sistemas, Tecnologia da informação
Centro Universitário Senac
Bacharelado, Ciência da Computação"""

#Exemplo Modelador 4
input_modelador_4 = """Contact
nononononon@mnonono.com
www.linkedin.com/in/nononononono
(LinkedIn)
Top Skills
Discovery
Liderança de equipe multifuncional
Gestão de produto técnico
Growth Hacking
Desenvolvimento de produtos
Gestão de produtos
Scrum
Product led Growth
Microsoft OfficeEdson Arantes do Nascimento (Pele)
Head of Product
Osasco, São Paulo, Brazil
Summary
Com mais de 15 anos de experiência em desenvolvimento de produtos digitais de alta complexidade, sou um líder de produtos apaixonado por inovação, dados e resultados.
Tenho um MBA em Gerenciamento de Projetos pela Fundação Getulio Vargas e diversas certificações em Design Thinking, Product Management e Yellow Belt.
Atualmente, sou Head of Products na ZAK, uma empresa líder em soluções para o setor de alimentação.
Sou responsável pela unidade de negócio de POS/PDV, onde gerencio equipes multidisciplinares que entregam produtos incríveis para clientes B2B e B2C.
Meu objetivo é sempre trabalhar com autonomia, alinhado aos objetivos estratégicos da empresa e buscando gerar valor real para o mercado. Estou constantemente motivado em buscar novos desafios e oportunidades de crescimento profissional.
Experience
Orizon
Group Product Manager
May 2024 - Present · 1 mo
São Paulo, Brazil
Zak
Head of Products
Jan 2022 - May 2024 · 2 yrs 5 mos
São Paulo, Brazil
Proudly responsible for the BU - POS/PDV
TC
Group Product Manager
May 2021 - Nov 2021 · 7 mos
Discovery e Growth - Produtos Private e Premium
Itau Unibanco
Group Product Manager
May 2020 - May 2021 · 1 yr 1 mo
São Paulo, Brasil
Projetos: Hipercard; Open Banking e PIX.
TOTVS
Lead Product Manager
Jan 2019 - May 2020 · 1 yr 5 mosJan 2019 to May 2020 · 1 yr 5 mos
São Paulo e Região, Brasil
Responsável pelo módulo financeiro com atividades estratégicas voltadas ao cliente e hub de inovação.
Projetos: CNAB Cloud, APP Viagens, Contas a Pagar e Receber e Inteligência Artificial Carol.
  Page 1 of 6

Evita dar feedback negativo, mesmo quando necessário para o desenvolvimento.
Elogia em excesso, sem apontar áreas de melhoria.
Desmotiva a equipe com feedbacks vagos e improdutivos.
Favorece apadrinhados e amigos, ignorando o talento e o mérito.
Cria um ambiente injusto e desmotivador.
Impede a retenção de talentos e o crescimento da equipe.
Education
FGV - Fundação Getulio Vargas
MBA - Gerente executivo de Projetos, Gerenciamento de Projeto
Anhanguera Educacional
Analista de Sistemas, Tecnologia da Informação"""



In [124]:

prompt_parts = [
  "Separe parte do textos de acordo com o contexto solicitado para organizar as informações respeitando somente os campos definidos no output Nome: ; Cargo: ; Resumo: ; Experiência Empresa 1: ; Cargo Empresa 1: ; ResumoCargo Empresa 1: ; Experiência Empresa 2: ; Cargo Empresa 2: ; ResumoCargo Empresa 2: ; Educação: ",
  f"input: {input_modelador_1}",
  "output: Nome: Fulano de Souza; Cargo: Backend Developer; Resumo: I am a passionate developer with 10 years of experience in developing robust and scalable applications, using technologies such as NodeJS, Python, MySQL, PostgreSQL, MongoDB, among others.Currently, I work on projects where I am responsible for creating and maintaining dynamic, scalable and secure web applications.My plurality of technologies is a strength that I have developed over the years, allowing me to contribute to the success of my clients and partners. I am always looking for new challenges and learning opportunities.; Experiência Empresa 1: Webcore Games; Cargo: Software Engineer; ResumoCargo Empresa 1: Desenvolvimento e manutenção de aplicações B2B e B2C, sendo a maioria projetadas para campanhas publicitárias com alto volume de usuários.; Experiência Empresa 2: Gorila; Cargo Empresa 2: Engenheiro de software; ResumoCargo Empresa 2: Trabalhei desenvolvendo as funcionalidades direcionadas aos clientes B2B, integrando com os outros serviços Gorila e externos.; Educação: Impacta Tecnologia",
  f"input: {input_modelador_2}",
  "output: Nome: Ciclana Moreira Silva; Cargo: Analista de marketing sênior; Resumo: ; Experiência Empresa 1: TOTVS; Cargo Empresa 1: Analista de marketing; ResumoCargo Empresa 1: ; Experiência Empresa 2: Sebrae-SP; Cargo Empresa 2: Analista comunicação e marketing; ResumoCargo Empresa 2: ; Educação: FIAP",
  f"input: {input_modelador_3}",
  "output: Nome: Markato do Brasil; Cargo: Front-end Developer | Angular; Resumo: Olá, sou Desenvolvedor Front-end graduado em Sistemas de Informação, com 4 anos de experiência em criação de sites.Acredito na importância da usabilidade e acessibilidade, por isso, me dedico a construir interfaces responsivas e intuitivas, sempre buscando a melhor experiência para o usuário.; Experiência Empresa 1: Lejour; Cargo Empresa 1: Desenvolvedor web; ResumoCargo Empresa 1: - Desenvolvimento de dashboards internos e externos utilizando Angular (sempre na última versão)- Participação na criação do design system com lib angular que contém e exporta estilos, componentes e serviços que serão reutilizados através dos diferentes dashboards- Desenvolvimento orientado a testes (TDD) em todos os projetos.; Experiência Empresa 2: Insize Brasil; Cargo Empresa 2: Auxiliar administrativo; ResumoCargo Empresa 2: ; Educação: Centro Universitário Senac",
  f"input: {input_modelador_4}",
  "output: Nome: Edson Arantes do Nascimento (Pele); Cargo: Head of Product; Resumo: Com mais de 15 anos de experiência em desenvolvimento de produtos digitais de alta complexidade, sou um líder de produtos apaixonado por inovação, dados e resultados. Tenho um MBA em Gerenciamento de Projetos pela Fundação Getulio Vargas e diversas certificações em Design Thinking, Product Management e Yellow Belt. Atualmente, sou Head of Products na ZAK, uma empresa líder em soluções para o setor de alimentação. Sou responsável pela unidade de negócio de POS/PDV, onde gerencio equipes multidisciplinares que entregam produtos incríveis para clientes B2B e B2C. Meu objetivo é sempre trabalhar com autonomia, alinhado aos objetivos estratégicos da empresa e buscando gerar valor real para o mercado. Estou constantemente motivado em buscar novos desafios e oportunidades de crescimento profissional.; Experiência Empresa 1: Orizon; Cargo Empresa 1: Group Product Manager; ResumoCargo Empresa 1: ; Experiência Empresa 2: Zak; Cargo Empresa 2: Head of Products; ResumoCargo Empresa 2: Proudly responsible for the BU - POS/PDV; Educação: FGV - Fundação Getulio Vargas",
  #f"input: {input_modelador_5}",
  #"output: Nome: ; Cargo: ; Resumo: ; Experiência Empresa 1: ; Cargo Empresa 1: ; ResumoCargo Empresa 1: ; Experiência Empresa 2: ; Cargo Empresa 2: ; ResumoCargo Empresa 2: ; Educação: "
  f"input: {input_avaliar}",
  "output: ", ]

response = model.generate_content(prompt_parts)
dado_estruturado = response.text
#print(dado_estruturado)

In [125]:
dado_estruturado = dado_estruturado.replace(",","\\")
response = model.generate_content(f"Transforme o resultado do texto abaixo em uma saida tipo csv {dado_estruturado} utilizando a estrutura de colunas Nome, Cargo, Resumo, Experiência Empresa 1,Cargo Empresa 1,ResumoCargo Empresa 1,Experiência Empresa 2,Cargo Empresa 2,ResumoCargo Empresa 2,Educação  com separação entre as colunas utilizando o caractere , e removendo linhas em branco")

dado_estruturado_tratado = response.text

dados_lista = dado_estruturado_tratado.split('\n')
dados_lista = [linha.split(',') for linha in dados_lista]

In [126]:
df = pd.DataFrame(dados_lista,columns=dados_lista[0])
df = df.drop(0)
df

,Nome,Cargo,Resumo,Experiência Empresa 1,Cargo Empresa 1,ResumoCargo Empresa 1,Experiência Empresa 2,Cargo Empresa 2,ResumoCargo Empresa 2,Educação
1,Themis Hoffmeister Villegas,Coordenador de Observabilidade\ Sustentação e ...,• Experiência na área de Tecnologia desde 1990...,credsystem,Coordenador de Observabilidade\ Sustentação e ...,,SysMap Solutions,Especialista em monitoramento,,UNINOVE


Analise do Perfil utilizando Gemini

In [127]:
nome_coluna = 'Cargo'
perfil_resumo_avaliado = ""

perfil_cargo = df['Cargo']
perfil_resumo = df['Resumo']

response = model.generate_content(f"Como um tech recruter avalie e compare com as melhores práticas e recomendações do mercado para um perfil no LinkedIn o campo Resumo, identifique o que não está adequado e recomende o que poderia ser incluido ou alterado. Considerando o cargo de {perfil_cargo}. Sendo o resumo {perfil_resumo}. Adicione no final uma tabela com duas colunas, sendo uma com o Resumo Informado e a outra coluna com o Resumo Sugerido pela analise." )
perfil_resumo_avaliado = response.text
#perfil_resumo_avaliado

In [128]:
nome_coluna = 'Cargo Empresa 1'
perfil_resumocargo_empresa_1_avaliado = ""

if nome_coluna in df.columns:
  perfil_cargo_empresa_1 = df['Cargo Empresa 1']
  perfil_resumocargo_empresa_1 = df['ResumoCargo Empresa 1']

  if perfil_cargo_empresa_1 is not None:
    response = model.generate_content(f"Como um tech recruter avalie e compare com as melhores práticas e recomendações do mercado para um perfil no LinkedIn o campo Descrição do campo de Experiência, , identifique o que não está adequado e recomende o que poderia ser incluido ou alterado. Considerando o cargo de {perfil_cargo_empresa_1}. Sendo o resumo {perfil_resumocargo_empresa_1}. Adicione no final uma tabela com duas colunas, sendo uma com o Resumo do Cargo Informado e a outra coluna com o Resumo do Cargo Sugerido pela analise.")
    perfil_resumocargo_empresa_1_avaliado = response.text
    #perfil_resumocargo_empresa_1_avaliado

In [129]:
nome_coluna = 'Cargo Empresa 2'
perfil_resumocargo_empresa_2_avaliado = ""

if nome_coluna in df.columns:
  perfil_cargo_empresa_2 = df['Cargo Empresa 2']
  perfil_resumocargo_empresa_2 = df['ResumoCargo Empresa 2']

  if perfil_cargo_empresa_2 is not None:
    response = model.generate_content(f"Como um tech recruter avalie e compare com as melhores práticas e recomendações do mercado para um perfil no LinkedIn o campo Descrição do campo de Experiência, , identifique o que não está adequado e recomende o que poderia ser incluido ou alterado. Considerando o cargo de {perfil_cargo_empresa_2}. Sendo o resumo {perfil_resumocargo_empresa_2}. Adicione no final uma tabela com duas colunas, sendo uma com o Resumo do Cargo Informado e a outra coluna com o Resumo do Cargo Sugerido pela analise.")
    perfil_resumocargo_empresa_2_avaliado = response.text
    #perfil_resumocargo_empresa_2_avaliado

In [130]:
#Dicas Gerais

response = model.generate_content(f"""Como um recrutador recomende as melhores práticas e dicas para melhorar a visibilidade do perfil no LinkedIn e aumentar as oportunidades de ser encontrado na rede. Detalhe os seguintes topicos
- Melhorar a foto do Perfil e informar o tamanho adequado em pixels
- Criar um Papel de Parede adequado e informar o tamanho adequado em pixels
- Recomendações dos top skill para {perfil_cargo}
- Sobre mais Experiencias e detalhe
- Certificações
- Projetos
- Recomendações""")
perfil_dicas_gerais = response.text

In [131]:
import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


display(to_markdown(perfil_resumo_avaliado))
print('-------------------------------------------')
if 'Cargo Empresa 1' in df.columns:
  display(to_markdown(perfil_resumocargo_empresa_1_avaliado))
  print('-------------------------------------------')
if 'Cargo Empresa 2' in df.columns:
  display(to_markdown(perfil_resumocargo_empresa_2_avaliado))
  print('-------------------------------------------')
display(to_markdown(perfil_dicas_gerais))
print('-------------------------------------------')


> **Avaliação do Resumo do LinkedIn**
> 
> **Resumo Informado:**
> 
> * **Experiência na área de Tecnologia desde 1990**
> 
> **Melhores Práticas e Recomendações:**
> 
> * O resumo deve ser conciso e impactante, destacando as habilidades e experiências mais relevantes para o cargo.
> * Deve incluir palavras-chave específicas relacionadas ao cargo para aumentar a visibilidade nos resultados da pesquisa.
> * Deve demonstrar valor quantificável e resultados alcançados.
> * Deve ser escrito em um tom profissional e envolvente.
> 
> **O que não está adequado:**
> 
> * O resumo fornecido é muito curto e não fornece informações suficientes sobre as habilidades e experiências do candidato.
> * Não inclui palavras-chave específicas relacionadas ao cargo de Coordenador de Observabilidade/Sustentação.
> * Não demonstra valor quantificável ou resultados alcançados.
> 
> **O que poderia ser incluído ou alterado:**
> 
> * **Habilidades:** Observabilidade, Sustentação, Monitoramento de Desempenho, Análise de Dados, Solução de Problemas
> * **Experiência:** Responsável por monitorar e manter a disponibilidade e o desempenho de sistemas de TI, identificar e resolver problemas de forma proativa, analisar dados para identificar tendências e melhorar a eficiência operacional
> * **Resultados:** Reduziu o tempo médio de reparo em 25% implementando um novo sistema de monitoramento e alertas
> * **Objetivos:** Buscar uma função de Coordenador de Observabilidade/Sustentação para utilizar minhas habilidades e experiência para otimizar a disponibilidade e o desempenho dos sistemas de TI
> 
> **Tabela Comparativa**
> 
> | Resumo Informado | Resumo Sugerido |
> |---|---|
> |   * Experiência na área de Tecnologia desde 1990 |   * Habilidades: Observabilidade, Sustentação, Monitoramento de Desempenho, Análise de Dados, Solução de Problemas   * Experiência: Responsável por monitorar e manter a disponibilidade e o desempenho de sistemas de TI, identificar e resolver problemas de forma proativa, analisar dados para identificar tendências e melhorar a eficiência operacional   * Resultados: Reduziu o tempo médio de reparo em 25% implementando um novo sistema de monitoramento e alertas   * Objetivos: Buscar uma função de Coordenador de Observabilidade/Sustentação para utilizar minhas habilidades e experiência para otimizar a disponibilidade e o desempenho dos sistemas de TI |

-------------------------------------------


> **Avaliação do Campo de Descrição de Experiência no LinkedIn**
> 
> **Cargo:** Coordenador de Observabilidade\ Sustentação e ...
> 
> **Descrição do Campo de Experiência:**
> 
> ```
> Name: Cargo Empresa 1, dtype: object.
> ```
> 
> **Melhores Práticas e Recomendações:**
> 
> * **Quantificar os resultados:** Fornecer dados específicos que demonstrem o impacto de suas contribuições.
> * **Usar palavras-chave relevantes:** Incluir termos de pesquisa comuns que os recrutadores usarão para encontrar candidatos qualificados.
> * **Destacar habilidades e responsabilidades essenciais:** Enfatizar as habilidades e responsabilidades mais relevantes para o cargo desejado.
> * **Personalizar para cada função:** Adaptar a descrição da experiência para destacar as qualificações mais relevantes para cada cargo específico.
> * **Manter a concisão:** Manter a descrição dentro de 200-300 palavras, focando nos pontos mais importantes.
> 
> **O que Não Está Adequado:**
> 
> * A descrição atual não quantifica os resultados nem usa palavras-chave relevantes.
> * Não destaca as habilidades e responsabilidades essenciais para o cargo de Coordenador de Observabilidade.
> * Não é personalizada para o cargo específico.
> 
> **Recomendações:**
> 
> * **Quantificar os Resultados:**
>     * "Implementei um sistema de monitoramento que reduziu o tempo de inatividade em 25%."
> * **Usar Palavras-Chave Relevantes:**
>     * "Observabilidade", "Sustentação", "Monitoramento de Infraestrutura", "Análise de Dados"
> * **Destacar Habilidades e Responsabilidades Essenciais:**
>     * "Monitorar e analisar métricas de desempenho de infraestrutura."
>     * "Identificar e resolver problemas de desempenho proativamente."
>     * "Colaborar com equipes de desenvolvimento e operações para melhorar a estabilidade do sistema."
> * **Personalizar para o Cargo Específico:**
>     * Adaptar a descrição para destacar as qualificações mais relevantes para o cargo específico.
> * **Manter a Concisão:**
>     * Resumir a descrição em 200-300 palavras, focando nos pontos mais importantes.
> 
> **Tabela de Resumo de Cargo**
> 
> | Resumo do Cargo Informado | Resumo do Cargo Sugerido |
> |---|---|
> | Name: Cargo Empresa 1, dtype: object. | Coordenador de Observabilidade com experiência comprovada em monitoramento de infraestrutura, análise de dados e resolução de problemas. Quantifiquei os resultados, implementando um sistema de monitoramento que reduziu o tempo de inatividade em 25%. Proficiente em ferramentas de observabilidade como Prometheus, Grafana e Splunk. |

-------------------------------------------


> **Avaliação do Campo de Descrição de Experiência**
> 
> **O que não está adequado:**
> 
> * **Falta de quantificação:** Não há números ou métricas específicas para demonstrar o impacto do trabalho do candidato.
> * **Linguagem passiva:** O uso de verbos passivos (por exemplo, "foi responsável por") enfraquece o impacto das realizações.
> * **Falta de detalhes:** A descrição é muito geral e não fornece informações suficientes sobre as responsabilidades e conquistas específicas do candidato.
> 
> **Recomendações para Inclusão ou Alteração:**
> 
> * **Quantificar realizações:** Use números e métricas para demonstrar o impacto quantificável do trabalho, como "reduziu o tempo de resposta em 20%" ou "aumentou a satisfação do cliente em 15%".
> * **Usar verbos ativos:** Use verbos ativos (por exemplo, "monitorou", "analisou") para destacar o papel ativo do candidato.
> * **Fornecer detalhes específicos:** Descreva as responsabilidades e conquistas específicas do candidato, incluindo ferramentas, tecnologias e metodologias usadas.
> * **Destacar habilidades e competências:** Mencione habilidades e competências relevantes para o cargo, como "monitoramento de rede", "análise de dados" e "resolução de problemas".
> 
> **Tabela de Resumo do Cargo**
> 
> | Resumo do Cargo Informado | Resumo do Cargo Sugerido |
> |---|---|
> | Responsável por monitorar e analisar sistemas de rede | Monitorou e analisou sistemas de rede, resultando em uma redução de 20% no tempo de resposta e um aumento de 15% na satisfação do cliente. |

-------------------------------------------


> **Melhores Práticas para Melhorar a Visibilidade do Perfil do LinkedIn**
> 
> **Melhorar a Foto do Perfil**
> 
> * Use uma foto profissional de alta qualidade que mostre claramente seu rosto.
> * Vista-se profissionalmente e certifique-se de que o fundo seja neutro.
> * O tamanho recomendado para a foto do perfil é 400 x 400 pixels.
> 
> **Criar um Papel de Parede Adequado**
> 
> * Escolha uma imagem de capa que represente seu setor ou área de especialização.
> * Use imagens de alta resolução que não sejam pixeladas.
> * O tamanho recomendado para a imagem de capa é 1584 x 396 pixels.
> 
> **Habilidades Recomendadas para Coordenador de Observabilidade/Sustentação**
> 
> * Monitoramento de Desempenho
> * Gerenciamento de Incidentes
> * Análise de Dados
> * Automação de TI
> * Gerenciamento de Alertas
> 
> **Sobre Mais Experiências e Detalhes**
> 
> * Descreva claramente suas responsabilidades e realizações em cada função que ocupou.
> * Use palavras-chave relevantes para o setor e as funções que você está buscando.
> * Quantifique seus resultados sempre que possível.
> 
> **Certificações**
> 
> * Liste todas as certificações relevantes que você possui.
> * Forneça links para credenciais e certificados verificados.
> 
> **Projetos**
> 
> * Destaque projetos pessoais ou profissionais que demonstrem suas habilidades e interesses.
> * Forneça descrições detalhadas e links para o trabalho concluído.
> 
> **Recomendações**
> 
> * Peça recomendações de colegas, gerentes ou clientes que possam atestar suas habilidades e ética de trabalho.
> * Certifique-se de que as recomendações sejam específicas e personalizadas.

-------------------------------------------
